In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-6-protein-function-prediction/IA.tsv
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo
/kaggle/input/protein-go-annotations-taxonomy/protein_go_annotations.csv
/kaggle/input/protein-go-annotations/goa_uniprot_all.csv


In [2]:
%pip install biopython

import pandas as pd
from collections import defaultdict
from Bio import SeqIO
import os

GOA_PATH  = "/kaggle/input/protein-go-annotations/goa_uniprot_all.csv"
OBO_PATH  = "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
TEST_FASTA = "/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta"

OUT_DIR  = "mydataset"
OUT_PATH = os.path.join(OUT_DIR, "blastfork.tsv")
os.makedirs(OUT_DIR, exist_ok=True)

# -------- LOAD GOA (remove NOT) --------
goa = pd.read_csv(GOA_PATH)
goa = goa[~goa["qualifier"].str.contains("NOT", na=False)]
goa = goa[["protein_id", "go_term"]].drop_duplicates()

# -------- LOAD TEST IDS --------
test_ids = set(r.id for r in SeqIO.parse(TEST_FASTA, "fasta"))

# -------- PARSE GO OBO (parents) --------
parents = defaultdict(set)
with open(OBO_PATH, "r") as f:
    cur = None
    for line in f:
        line = line.strip()
        if line == "[Term]":
            cur = None
        elif line.startswith("id: "):
            cur = line.split("id: ")[1]
        elif cur and line.startswith("is_a: "):
            parents[cur].add(line.split()[1])
        elif cur and line.startswith("relationship: part_of "):
            parents[cur].add(line.split()[2])

def get_ancestors(go_id):
    out, stack = set(), [go_id]
    while stack:
        g = stack.pop()
        for p in parents.get(g, []):
            if p not in out:
                out.add(p)
                stack.append(p)
    return out

ROOTS = {"GO:0008150", "GO:0003674", "GO:0005575"}
rows = []

blast = goa[goa["protein_id"].isin(test_ids)]

for pid, g in blast.groupby("protein_id"):
    terms = set(g.go_term)
    for t in list(terms):
        terms |= get_ancestors(t)
    for t in terms:
        if t not in ROOTS:
            rows.append((pid, t, 1.0))

blastfork = pd.DataFrame(rows, columns=["Id", "GO term", "Confidence"])

blastfork.to_csv(OUT_PATH, sep="\t", index=False, header=False)

print("blastfork.tsv created")
print(f"Path: {OUT_PATH}")
print(f"Rows: {len(blastfork)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
blastfork.tsv created
Path: mydataset/blastfork.tsv
Rows: 11865250
